In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import sys
import math
sys.path.append('/home/eduardo/PycharmProjects/treemap-analysis/code')

import Parser
import SpatialMetrics
import TemporalMetrics

In [ ]:
dataset_id = 'exports'

In [ ]:
df = SpatialMetrics.aggregated_ar('git', dataset_id)
n_revisions = df.shape[0]

data = []
for r in df.as_matrix():
    data.append(r[~np.isnan(r)])

In [ ]:
def styleBoxplot(bp, ax):
    # change outline color, fill color and linewidth of the boxes
    for box in bp['boxes']:
        # change outline color
        box.set(color='#AAAAAA', linewidth=.1)
        # change fill color
        box.set(facecolor='#AAAAAA')
    for median in bp['medians']:
        median.set(color='#000000', linewidth=1)
    for whisker in bp['whiskers']:
        whisker.set(color='#666666', linestyle='-')
    for cap in bp['caps']:
        cap.set(color='#FFFFFF', linewidth=0)
        
    # Set only 3 ticks
    ax.set_xticks([1, n_revisions / 2, n_revisions], minor=False)
    ax.set_xticklabels([1, int(n_revisions / 2), n_revisions], fontdict=None, minor=False)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.tick_params(axis='x', which='both', top='off')
    ax.tick_params(axis='y', which='both', right='off', left='off')

In [ ]:
fig = plt.figure()
bp = plt.boxplot(data, whis=[5, 95], showfliers=False, patch_artist=True, meanline=True, widths = 1);
styleBoxplot(bp, plt.gca())
plt.title('git')

In [ ]:
def cm2inch(value):
    return value/2.54

# Aspect ratio boxplots

In [ ]:
technique_list = Parser.list_techniques()
nrow = 7
ncol = 2
fig, axs = plt.subplots(nrow, ncol, sharex=True, sharey=True, figsize=(cm2inch(20), cm2inch(2*22)))
fig.delaxes(axs[6, 1])
for i, ax in enumerate(fig.axes):
    if i < len(technique_list):
        ax.set_title(technique_list[i])
        df = SpatialMetrics.aggregated_ar(technique_list[i], dataset_id)

        data = []
        for r in df.as_matrix():
            data.append(r[~np.isnan(r)])

        bp = ax.boxplot(data, whis=[5, 95], showfliers=False, patch_artist=True, meanline=True);
        styleBoxplot(bp, ax)
        
        ax.set_ylim([0, .75])
        ax.set_yticks([0, .25, .5, .75], minor=False)
        ax.set_yticklabels([0, .25, .5, .75], fontdict=None, minor=False)

fig.tight_layout()

# Stability boxplots
Unweighted Pearson correlation coefficient in red

In [ ]:
technique_list = Parser.list_techniques()
nrow = 7
ncol = 2
fig, axs = plt.subplots(nrow, ncol, sharex=True, sharey=True, figsize=(cm2inch(20), cm2inch(2*22)))
fig.delaxes(axs[6, 1])
for i, ax in enumerate(fig.axes):
    ax.set_title(technique_list[i])
    df, pearson, weighted_pearson = TemporalMetrics.aggregated_stab(technique_list[i], dataset_id)

    data = []
    for r in df.as_matrix():
        data.append(r[~np.isnan(r)])

    bp = ax.boxplot(data, whis=[5, 95], showfliers=False, patch_artist=True, meanline=True);
    styleBoxplot(bp, ax)
    
    x = [i for i in range(len(pearson))]
    ax.plot(x, pearson, 'r_', ms=4)
     
fig.tight_layout()